In [158]:
import cv2
cv2.__version__

'4.5.1'

In [159]:
#program deteksi objek

import matplotlib.pyplot as plt
import cv2
import numpy as np
import math

box=[]

# Load Yolo
net = cv2.dnn.readNet("yolov4-obj_best.weights","yolov4-obj.cfg")
#net = cv2.dnn.readNet("20_yolov3.weights","20_yolov3.cfg")
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

color_blue = (255,0,0)
color_red = (0,0,255)
#colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Loading image
image = cv2.imread("sampelGambar/SUNNY/camera8.jpg")
#img = cv2.resize(imges, None, fx=1, fy=1)
height, width, channels = image.shape

# Detecting objects
#blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
blob = cv2.dnn.blobFromImage(image, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.1:
            if class_id == 1:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.2)

#print(indexes)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        
        box.append(boxes[i])
        #print(boxes[i])
        label = str(classes[class_ids[i]])
        #color = colors[i]
        cv2.rectangle(image, (x, y), (x + w, y + h), color_blue, 2)
        cv2.putText(image, label, (x, y + 30), font, 2, color_blue, 2)
        cv2.putText(image, str("{:.2f}".format(confidences[i])), (x, y + 50), font, 2, color_red, 2)
        
cv2.imshow("output1",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
#cv2.imwrite("output1.jpg", image)


In [160]:
#Load koodinat npy
ROI_slot = np.load('coordinate/cam8.npy',allow_pickle=True)

In [161]:
ROI_slot[0][0]

[[967, 73], [995, 101], [947, 101], [922, 73]]

In [162]:
boxess = []
for i in range(len(indexes)):
    boxess.append([])
    boxess[i].append(box[i][0])
    boxess[i].append(box[i][1])
    boxess[i].append(box[i][2]+box[i][0])
    boxess[i].append(box[i][3]+box[i][1])

In [163]:
ROI_car = []
for i in range(len(indexes)):
    ROI_car.append([(boxess[i][0],boxess[i][3]), (boxess[i][0],boxess[i][1]), (boxess[i][2],boxess[i][1]), (boxess[i][2],boxess[i][3])])    


In [164]:
#inisialisasi array
diagonal_slot = []                      
centroid_slot_x = []
centroid_slot_y = []
for i in range (len(ROI_slot)):            
    new = []
    new1 = []
    new2 = []
    new3 = [0,0]
    for j in (ROI_slot[i]):
        new.append(0)
        new1.append(0)
        new2.append(0)
    diagonal_slot.append(new)    
    centroid_slot_x.append(new1)
    centroid_slot_y.append(new2)

    
#print(centroid_slot)
#print(diagonal_slot)
    
for row in range(len(ROI_slot)):
    for slots in range(len(ROI_slot[row])):
        x1 = ROI_slot[row][slots][0][0]
        y1 = ROI_slot[row][slots][0][1]
        x2 = ROI_slot[row][slots][1][0]
        y2 = ROI_slot[row][slots][1][1]
        x3 = ROI_slot[row][slots][2][0]
        y3 = ROI_slot[row][slots][2][1]
        x4 = ROI_slot[row][slots][3][0]
        y4 = ROI_slot[row][slots][3][1]
        #menghitung centroid slot        
        centroid_slot_x[row][slots] = (x3+x1)/2
        centroid_slot_y[row][slots] = (y3+y1)/2        
        
        #print((x3+x1)/2)
        #menghitung diagonal slot
        d1 = math.sqrt(((y3-y1)**2)+((x3-x1)**2))
        d2 = math.sqrt(((y4-y2)**2)+((x4-x2)**2))
        if d1<d2:
            diagonal_slot[row][slots] = d1
        else:
            diagonal_slot[row][slots] = d2
        


In [165]:
#Load images
image = cv2.imread("sampelGambar/SUNNY/camera8.jpg")
image = cv2.resize(image, (image.shape[1],image.shape[0]), interpolation = cv2.INTER_AREA)
plottedImage = image.copy()

In [166]:
#Menghitung ED
carX = 0
carY = 0
allCentroidCar = []
ED = []
perc_ED = []
Status = []
min_ed_car = []
car_id = []
count = 0
idx_and_car = []
idx_car_list = []

#car_idx = []
car_idx =  []
for i in range (len(ROI_slot)):            
    new = []
    new1 = []
    for j in (ROI_slot[i]):
        new.append(0)
        new1.append(0)
    ED.append(new)
    perc_ED.append(new)
    Status.append(new1)

#gambar titik centroid di mobil
for car in range(len(ROI_car)):
    x1 = ROI_car[car][0][0]
    y1 = ROI_car[car][0][1]
    x3 = ROI_car[car][2][0]
    y3 = ROI_car[car][2][1]
    centroid_car_x = (x1+x3)/2
    centroid_car_y = (y1+y3)/2
    allCentroidCar.append((int(centroid_car_x),int(centroid_car_y)))
    mod = cv2.circle(plottedImage, (int(centroid_car_x),int(centroid_car_y)), 4, (0,0,255), -1)

for row in range(len(ROI_slot)):
    for slots in range(len(ROI_slot[row])):
        centroid_slot_xx = centroid_slot_x[row][slots]
        centroid_slot_yy = centroid_slot_y[row][slots]
        #titik di slot
        mod = cv2.circle(plottedImage, (int(centroid_slot_xx),int(centroid_slot_yy)), 4, (0,0,255), -1)
        temp = []
        temp2 = []
        for car in range(len(ROI_car)):
            x1 = ROI_car[car][0][0]
            y1 = ROI_car[car][0][1]
            x3 = ROI_car[car][2][0]
            y3 = ROI_car[car][2][1]
              
            centroid_car_x = (x1+x3)/2
            centroid_car_y = (y1+y3)/2
            
            temp.append(math.sqrt(((centroid_slot_yy-centroid_car_y)**2)+((centroid_slot_xx-centroid_car_x)**2)))
            temp2.append(car)
        
        min_ed = min(temp)
        
        ED[row][slots] = min(temp)
        perc_ED[row][slots] = (ED[row][slots]/diagonal_slot[row][slots])*100
        if perc_ED[row][slots] < 75:
            Status[row][slots] = 'OCCUPIED'
        else:
            Status[row][slots] = 'VACANT'           
                      
        idx = temp.index(min(temp))
        idx_car = temp2[idx]
        idx_car_list.append(idx_car)      
        min_ed_car.append([idx_car, row, slots, min_ed])


x = [min_ed_car[i][0] for i in range (len(min_ed_car))]
h = max(idx_car_list)

for count in range (h+1):
    tempp = []
    slott = []
    for i in range (len(x)):
        if (x[i] in x[:i]) or (x[i] in x[i+1:]):        
            if x[i]==count:
                tempp.append(min_ed_car[i][3])
                slott.append([min_ed_car[i][1],min_ed_car[i][2]])   
    if not tempp:
        minn = 99999
    else:
        minn = min(tempp)             
        indx = tempp.index(min(tempp))
        for i in range (len(slott)):
            if (i == indx):
                ED[slott[i][0]][slott[i][1]] = minn
                perc_ED[slott[i][0]][slott[i][1]] = (ED[slott[i][0]][slott[i][1]]/diagonal_slot[slott[i][0]][slott[i][1]])*100
                if perc_ED[slott[i][0]][slott[i][1]] < 75:
                    Status[slott[i][0]][slott[i][1]] = 'OCCUPIED'
                else:
                    Status[slott[i][0]][slott[i][1]] = 'VACANT'
            else:
                Status[slott[i][0]][slott[i][1]] = 'VACANT'
                
fontScale = 0.8

for row in range(len(ROI_slot)):
    for slots in range(len(ROI_slot[row])):        
        #draw ROI_slot
        mod = cv2.polylines(plottedImage, np.int32([ROI_slot[row][slots]]), True, (255,0,0), thickness=2)
        #draw Status_slot
        mod = cv2.putText(plottedImage, Status[row][slots], (int(centroid_slot_x[row][slots])-10, int(centroid_slot_y[row][slots])),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 0), 1, cv2.LINE_AA)
       
    
    
        #Text slot
        #PENTING extY GANTI KE '0' UNTUK CAMERA 8 DAN '15' UNTUK CAMERA 1. BIAR TIDAK TUMPANG TINDIH
        extY = 0
        minROISlot = min(ROI_slot[row][slots])
        org = (minROISlot[0], minROISlot[1]+extY)
        slotId = "("+str(row)+"-"+str(slots)+")"
        cv2.putText(plottedImage, slotId, org, font, fontScale, (255,255,255), 1)

        


In [167]:
#DRAW LINE
distancez = []
def euclideanDistance(coordinate1, coordinate2):
    return pow(pow(coordinate1[0] - coordinate2[0], 2) + pow(coordinate1[1] - coordinate2[1], 2), .5)

for row in range(len(ROI_slot)):
    for slots in range(len(ROI_slot[row])):
        distancez = []
        if Status[row][slots] == 'VACANT':
            continue
        else:
            centSlotX = int(centroid_slot_x[row][slots])
            centSlotY = int(centroid_slot_y[row][slots])
            for i in allCentroidCar:
                distancez.append(euclideanDistance((centSlotX,centSlotY),(i[0],i[1])))
            minDistancezIndex = distancez.index(min(distancez))
            centCarX = allCentroidCar[minDistancezIndex][0]
            centCarY = allCentroidCar[minDistancezIndex][1]
            mod = cv2.line(plottedImage,(centSlotX,centSlotY),(centCarX,centCarY),(0,255,255),2) 

In [168]:
cv2.imshow('Shapes', mod)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [169]:
cv2.imwrite('27April/Hasil_ED/SUNNY/camera8.jpg',mod)

True